# Stable Diffusion Outpainting 

In [ ]:
!pip install diffusers transformers accelerate


In [ ]:
import PIL
import requests
import torch
from io import BytesIO
from PIL import Image
from diffusers import StableDiffusionInpaintPipeline
import numpy as np

In [ ]:
from downscale_and_noise import get_init_mask_image

with open('./images/base.png', 'rb') as f:
    base_image = Image.open(f)
    base_image.load()

init_image, mask_image = get_init_mask_image(base_image)
init_image_arr = np.array(init_image)
mask_image_arr = np.array(mask_image)


In [ ]:
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting"
).to('cuda')
pipe.enable_attention_slicing()

In [ ]:
prompt = "A dog sitting on a park bench"
outpainted_image = pipe(prompt=prompt, image=init_image_arr, mask_image=mask_image_arr).images[0]

In [ ]:
prompt = "A dog sitting on a park bench"
NB_STEPS = 10
for i in range(NB_STEPS):
  print('STEP: ', i)
  cur_image = pipe(prompt=prompt, image=Image.fromarray(init_image), mask_image=Image.fromarray(mask_image)).images[0]
  # todo rendre trensparent les pixels du masque de l'image précédente
  cur_image.save(f'./save/{str(i)}.png')
  init_image, mask_image = get_init_mask_image(cur_image)


In [ ]:
!zip -r frames.zip save/*